# Tugas 1 Seleksi Lab Basis Data - Data Scraping and Storing

## Website: 
https://www.dotproperty.id/properties-for-rent : Data properti yang disewakan.
<br>https://www.dotproperty.id/properties-for-sale : Data properti yang dijual

## Nama: Muhammad Zaky
## NIM: 18220071

## Import Required Libraries

In [1]:
import json
from bs4 import BeautifulSoup
import requests
import time
import os

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

## Pendefinisian Beberapa Fungsi

### Permasalahan

- Attribute Luas Tanah memiliki tanda ',' sebagai pemisah angka ribuan dan di dalam attribute luas tanah juga terdapat satuan (m2) yang tidak kita butuhkan
- Attribute Harga memiliki dua tipe, yaitu yang berbentuk jutaan dan miliar. 
<br> - Untuk attribute Harga yang bertipe jutaan, attribute ini masih memiliki satuan 'Rp' dan dipisahkan oleh tanda ',' (Contoh: Rp 200,000,000).
<br> - Untuk attribute Harga yang bertipe miliar, attribute ini dalam bentuk satuan 'Miliar' dan masih memiliki satuan 'Rp' (Contoh: Rp 2.85 Miliar).
- Attribute Lokasi kita ingin pecah menjadi kota dan provinsi karena rata-rata dari attribute ini memiliki format seperti ({Nama Kota}, {Nama Provinsi}), sehingga ini akan lebih baik untuk dipecah menjadi kota dan provinsi.

### Fungsi Pendukung

In [3]:
def extract_luas_tanah(luas_tanah): #Fungsi untuk mengambil angka dari 'luas tanah' dan membentuknya dalam format data float.
    luas = luas_tanah.split(' ')
    if(luas[0].__contains__(',')):
        return luas[0].replace(',','')
    return float(luas[0])

def extract_harga_juta(harga): #Fungsi untuk mengambil angka dari 'harga' yang bertipe jutaan, sehingga sudah tidak ada satuan 'Rp' dan tidak dipisahkan oleh tanda ','.
    hargasplit = harga.split(' ')
    return float(hargasplit[1].replace(',', ''))

def extract_harga_miliar(harga): #Fungsi untuk mengambil angka dari 'harga' yang bertipe miliar, sehingga sudah tidak ada satuan 'Rp'.
    hargasplit = harga.split(' ')
    return float(hargasplit[1])

def extract_kota(lokasi): #Fungsi untuk mengambil nama kota dari 'lokasi'.
    return lokasi.split(',')[0]

def extract_provinsi(lokasi): #Fungsi untuk mengambil nama provinsi dari 'lokasi' dan menghilangkan spasi yang berada di awal.
    return (lokasi.split(',')[1])[1:]

### Fungsi Utama

In [4]:
def scraping_pages(data, object, firstPage, lastPage): #Fungsi utama untuk melakukan scraping yang menerima masukan 'data' berupa ['rent', 'sale'] sebagai parameter searching.
    count_pages = 1 #Menghitung jumlah halaman yang sudah di scrape.
    for page in range(firstPage, lastPage+1):
        n = 0 #Untuk menghitung banyak data yang di scrape dari sebuah halaman.
        template = "https://www.dotproperty.id/properties-for-{}?page="
        pages = requests.get(template.format(data)+"?page="+str(page), headers=headers)
        soup = BeautifulSoup(pages.text, 'html.parser')
        properti = soup.find_all(True, {'class':['search-list normal', 'search-list featured', 'search-list premium']})
        for item in properti:
            tag = item.find('div', 'right-block')
            
            #Title dari properti.
            title = item.a.get('title')
            
            #Link dari properti
            link = item.a.get('href')
            
            #Harga dan unit harga dari properti.
            harga = tag.find('div', 'price').text.strip()
            unit = ''
            try:
                if (harga.__contains__('Milyar')):
                    unit = 'Miliar'
                    hargaint = extract_harga_miliar(harga) #harga dalam bentuk float (Contoh: 2.85)
                    harga_idr = int(hargaint*10**9) #harga dalam bentuk rupiah secara utuh (Contoh: 2850000000)
                else:
                    unit = 'Juta'
                    hargaint = extract_harga_juta(harga)/10**6
                    harga_idr = int(extract_harga_juta(harga))
            except ValueError:
                unit = ''
                hargaint = ''
            
            #Badge dari properti ['Normal', 'Utama', 'Premium']
            try:
                badge = item.find('div', 'rectangle-badge').text.strip()
            except AttributeError:
                badge = 'Normal'
            
            #Nama dari properti
            nama = tag.h3.text.strip()
            
            #Lokasi dari properti yang dipecah menjadi kota dan provinsi
            lokasi = tag.find('div', 'location').text.strip()
            if (lokasi.__contains__(',')):
                kota = extract_kota(lokasi)
                provinsi = extract_provinsi(lokasi)
            else:
                kota = ''
                provinsi = extract_kota(lokasi)
                
            #Tipe properti
            tipe = tag.find('div', 'hidden-property-type hidden').text
            
            #Accomodation(Kamar mandi, Kamar tidur, Luas tanah)
            ul = tag.find('ul')
            acc = []
            for li in ul.find_all('li'):
                acc.append(li.span.text)
            if(len(acc) == 3):
                kamar_mandi = int(acc[0])
                kamar_tidur = int(acc[1])
                luas_tanah = acc[2]
                luas_int = extract_luas_tanah(luas_tanah)
            else:
                acckamar = []
                for item in acc:
                    if(item.__contains__('m2')):
                        luas_tanah = item
                        luas_int = extract_luas_tanah(luas_tanah)
                    else:
                        acckamar.append(item)
                        luas_tanah = ''
                if(len(acckamar) == 2):
                    kamar_tidur = int(acckamar[0])
                    kamar_mandi = int(acckamar[1])
                elif(len(acckamar) == 1):
                    kamar_tidur = int(acckamar[0])
                    kamar_mandi = ''
                elif(len(acckamar) == 0):
                    kamar_tidur = ''
                    kamar_mandi = ''
            
            #Status apakah disewakan atau dijual
            status = ''
            if(data == 'rent'):
                status = 'Disewakan'
            elif(data == 'sale'):
                status = 'Dijual'
            
            pembayaran = ''
            if (status == 'Disewakan'):
                pembayaran = "Per "+tag.small.text.strip()

            value = {
                "Title": title, 
                "Nama": nama,
                "Badge": badge,
                "Tipe": tipe,
                "Jumlah Kamar Mandi": kamar_mandi,
                "Jumlah Kamar Tidur": kamar_tidur, 
                "Luas Tanah(m2)": luas_int,
                "Harga (IDR)": harga_idr,
                "Harga (float)": hargaint,
                "Unit Harga": unit,
                "Status Pembayaran": pembayaran,
                "Kota": kota,
                "Provinsi": provinsi,
                "Status": status,
                "Link": link
                    }
            object.append(value)
            n += 1
        print("Berhasil melakukan scraping halaman ke-"+str(count_pages)+" sebanyak "+str(n)+" data.")
        count_pages += 1
        time.sleep(1) #Agar tidak diblock

In [5]:
#Scrape data properti yang disewakan
rentdata = []
scraping_pages('rent', rentdata, 1, 50)

Berhasil melakukan scraping halaman ke-1 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-2 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-3 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-4 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-5 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-6 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-7 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-8 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-9 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-10 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-11 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-12 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-13 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-14 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-15 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-16 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-17 sebanya

In [6]:
#Scrape data properti yang dijual
saledata = []
scraping_pages('sale', saledata, 1, 50)

Berhasil melakukan scraping halaman ke-1 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-2 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-3 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-4 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-5 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-6 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-7 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-8 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-9 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-10 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-11 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-12 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-13 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-14 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-15 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-16 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-17 sebanya

In [8]:
rentsaledata = rentdata + saledata

In [9]:
filename = input("Nama File: ")
filepath = 'G:/My Drive/Tugas Lab Basis Data/Seleksi-2022-Tugas-1/Data Scraping/data'
dump = json.dumps(rentsaledata, indent=4)
filepath = os.path.join(filepath, filename + ".json")
with open(filepath, "w") as file:
    file.write(dump)
    file.close()

Nama File:  Rent and Sale Properties Data


In [10]:
len(rentsaledata)

2500